# Comparing line-of-sight CO2 with experiments

Here we use a line-of-sight model with 3 slabs : a  high temperature CO2 emission at 1450 K, a post-discharge reactor at 350 K, and room absorption with 400 ppm. 

---

In [1]:
# Initialize session
%matplotlib widget

### Compute CO2 spectra

We use [calc_spectrum](https://radis.readthedocs.io/en/latest/source/radis.lbl.calc.html#radis.lbl.calc.calc_spectrum) which will conveniently download data from HITRAN. Note that for the high temperatures considered here, we should instead of use HITEMP. This is possible on a local installation of RADIS : see the [documentation](https://radis.readthedocs.io) 

In [2]:
from radis import calc_spectrum
from astropy import units as u

s = calc_spectrum(4080 * u.nm, 4500 * u.nm,
                  molecule='CO2',
                  isotope='1,2,3',
                  pressure=1.01325 * u.bar,
                  Tgas=1450 * u.K,
                  mole_fraction=0.5,
                  path_length=0.11 * u.cm,
                  name='Plasma',
                  verbose=False,
                  )
s.plot('radiance_noslit', wunit='nm')

/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/misc/warning.py:241: HighTemperatureWarning: HITRAN is valid for low temperatures (typically < 700 K). For higher temperatures you may need HITEMP or CDSD. See the 'databank=' parameter
  warnings.warn(WarningType(message))
/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/misc/warning.py:241: MissingSelfBroadeningWarning: Self-broadening temperature coefficient Tdpsel not given in database: used Tdpair instead
  warnings.warn(WarningType(message))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
s_post = calc_spectrum(4080 * u.nm, 4500 * u.nm,
                  molecule='CO2',
                  isotope='1,2,3',
                  pressure=1.01325 * u.bar,
                  Tgas=350 * u.K,
                  mole_fraction=1.0,
                  path_length=5.5 * u.cm,
                  name='Post',
                  verbose=False,
                  )
s.plot('transmittance_noslit', wunit='nm')

/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/misc/warning.py:241: MissingSelfBroadeningWarning: Self-broadening temperature coefficient Tdpsel not given in database: used Tdpair instead
  warnings.warn(WarningType(message))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
s0 = calc_spectrum(4080 * u.nm, 4500 * u.nm,
                  molecule='CO2',
                  isotope='1,2,3',
                  pressure=1.01325 * u.bar,
                  Tgas=300 * u.K,
                  mole_fraction=400e-6,
                  path_length=300 * u.cm,
                  name='Room',
                  verbose=False
                  )
s0.plot('transmittance_noslit', wunit='nm')

/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/misc/warning.py:241: MissingSelfBroadeningWarning: Self-broadening temperature coefficient Tdpsel not given in database: used Tdpair instead
  warnings.warn(WarningType(message))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Line-of-sight spectrum

To compute the line-of-sight we use [SerialSlabs](https://radis.readthedocs.io/en/latest/source/radis.los.slabs.html#radis.los.slabs.SerialSlabs) from the [line-of-sight module](https://radis.readthedocs.io/en/latest/los/los.html). Note that `SerialSlabs(s, s0)` is equivalent to `s > s0` but this syntax is limited to two spectra only. 

In [5]:
from radis import SerialSlabs
s_los = SerialSlabs(s, s_post, s0)


### Instrumental slit function 

We then apply an instrumental slit function to the line-of-sight spectrum. We can use a trapezoidal slit function with FWHM~1.6 nm. As shown below, this corresponds quite well to the instrumental slit function measured experimentally, and shown with the convenient [plot_slit](https://radis.readthedocs.io/en/develop/source/radis.tools.slit.html#radis.tools.slit.plot_slit) function. 


In [6]:
s_los.apply_slit((1.55, 1.65), 'nm', shape='trapezoidal')
s_los.plot_slit()

from radis import plot_slit
plot_slit("slit_function.txt")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 1 Axes>,
 <AxesSubplot:xlabel='Wavespace', ylabel='Slit function'>)

We could also have used an experimentally measured slit function directly by giving the filename to [apply_slit()](https://radis.readthedocs.io/en/latest/source/radis.spectrum.spectrum.html#radis.spectrum.spectrum.Spectrum.apply_slit) . 

[apply_slit()](https://radis.readthedocs.io/en/latest/source/radis.spectrum.spectrum.html#radis.spectrum.spectrum.Spectrum.apply_slit) also allows us to correct for the change of the slit function in the wavelength space, which results from the pixel to wavelength calibration function and changes with the wavelength due to the linear dispersion of the spectrometer. RADIS features an easy way to compute and correct for this slit dispersion : 

In [7]:
from radis.test.tools.test_slit import linear_dispersion
dispersion = lambda w: linear_dispersion(w, 756, phi=-6.91, m=1, gr=297.42) # Reciprocal linear dispersion of the Acton 750i
s_los.apply_slit("slit_function.txt", slit_dispersion=dispersion)

Resampling - Energy conservation: 100.01%
4430.11 to 4489.65nm: slit function FWHM changed from 1.64 to 1.62
4430.11 to 4457.75nm: slit function FWHM changed from 1.64 to 1.63
4430.11 to 4412.99nm: slit function FWHM changed from 1.64 to 1.65
4430.11 to 4366.59nm: slit function FWHM changed from 1.64 to 1.67
4430.11 to 4318.41nm: slit function FWHM changed from 1.64 to 1.68
4430.11 to 4268.37nm: slit function FWHM changed from 1.64 to 1.70
4430.11 to 4216.36nm: slit function FWHM changed from 1.64 to 1.72
4430.11 to 4162.25nm: slit function FWHM changed from 1.64 to 1.74
4430.11 to 4108.50nm: slit function FWHM changed from 1.64 to 1.76


/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/tools/slit.py:907: SlitDispersionWarning: Slit dispersion changes slightly (1.08%) between 4434.243 and 4481.499nm. Consider splitting your spectrum
  SlitDispersionWarning,
/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/tools/slit.py:907: SlitDispersionWarning: Slit dispersion changes slightly (1.09%) between 4388.744 and 4437.529nm. Consider splitting your spectrum
  SlitDispersionWarning,
/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/tools/slit.py:907: SlitDispersionWarning: Slit dispersion changes slightly (1.09%) between 4341.509 and 4391.963nm. Consider splitting your spectrum
  SlitDispersionWarning,
/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/tools/slit.py:907: SlitDispersionWarning: Slit dispersion changes slightly (1.10%) between 4292.479 and 4344.659nm. Consider splitting your spectrum
  SlitDispersionWarning,
/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/tools/sli

## Compare with experiment

### Experiment spectrum

We load the experiment file in `radis` [Spectrum](https://radis.readthedocs.io/en/latest/source/radis.spectrum.spectrum.html#radis.spectrum.spectrum.Spectrum) object directly from the file. We could also import it with `numpy` and use [Spectrum.from_array()](https://radis.readthedocs.io/en/latest/source/radis.spectrum.spectrum.html#radis.spectrum.spectrum.Spectrum.from_array). We could have also loaded a radis `.spec` file (with units and metadata) directly with [load_spec](https://radis.readthedocs.io/en/latest/source/radis.tools.database.html#radis.tools.database.load_spec)

In [8]:
from radis import Spectrum
# Import an experimental spectrum : 
s_exp = Spectrum.from_txt('experiment.txt', 'radiance', unit='µW/cm2/sr/nm', waveunit='nm')       

### Compare

We use [plot_diff()](https://radis.readthedocs.io/en/latest/source/radis.spectrum.compare.html#radis.spectrum.compare.plot_diff) 

In [9]:
from radis import plot_diff
plot_diff(s_exp, s_los)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 2 Axes>,
 [<AxesSubplot:>, <AxesSubplot:xlabel='Wavelength (nm)'>])

## Analysis

### Post-discharge absorption

Let's compare the emission of the hot CO2 to the total line-of-sight emission. For visualization purpose we apply the same slit function to the hot CO2 slab.   

In [10]:
s.apply_slit("slit_function.txt", slit_dispersion=dispersion)
s.plot(wunit='nm')
             
s_los.plot(wunit='nm', nfig='same')

#import matplotlib.pyplot as plt
#plt.xlim((4165, 4180))

Resampling - Energy conservation: 100.01%
4430.11 to 4489.65nm: slit function FWHM changed from 1.64 to 1.62
4430.11 to 4457.75nm: slit function FWHM changed from 1.64 to 1.63
4430.11 to 4412.99nm: slit function FWHM changed from 1.64 to 1.65
4430.11 to 4366.59nm: slit function FWHM changed from 1.64 to 1.67
4430.11 to 4318.41nm: slit function FWHM changed from 1.64 to 1.68
4430.11 to 4268.37nm: slit function FWHM changed from 1.64 to 1.70
4430.11 to 4216.36nm: slit function FWHM changed from 1.64 to 1.72
4430.11 to 4162.25nm: slit function FWHM changed from 1.64 to 1.74
4430.11 to 4108.50nm: slit function FWHM changed from 1.64 to 1.76


/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/tools/slit.py:907: SlitDispersionWarning: Slit dispersion changes slightly (1.08%) between 4434.243 and 4481.499nm. Consider splitting your spectrum
  SlitDispersionWarning,
/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/tools/slit.py:907: SlitDispersionWarning: Slit dispersion changes slightly (1.09%) between 4388.744 and 4437.529nm. Consider splitting your spectrum
  SlitDispersionWarning,
/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/tools/slit.py:907: SlitDispersionWarning: Slit dispersion changes slightly (1.09%) between 4341.509 and 4391.963nm. Consider splitting your spectrum
  SlitDispersionWarning,
/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/tools/slit.py:907: SlitDispersionWarning: Slit dispersion changes slightly (1.10%) between 4292.479 and 4344.659nm. Consider splitting your spectrum
  SlitDispersionWarning,
/srv/conda/envs/notebook/lib/python3.6/site-packages/radis/tools/sli

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The emission is absorbed even in the bandhead at 4.2 µm, as can be shown by uncommenting the xlim() parameters in the cell above to zoom on the 4165 - 4180 nm region. The emission is mostly absorbed by the post-discharge slab, as shown by plotting the transmittance below, which is less than 0.8 at the bandhead :  

In [11]:
s_post.plot('transmittance_noslit', wunit='nm')

import matplotlib.pyplot as plt
plt.xlim((4165, 4180))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(4165.0, 4180.0)

This emphasizes the importance of computing large lineshapes. Here, even a lineshape cutoff parameter of 20 cm-1 would not be enough to account for the bandhead absorption. 

It is also known that far from the line center, the CO2 lineshape does not follow a perfect Voigt profile. This is not taken into account in RADIS 0.9.26. Advanced lineshapes require to take into account speed-dependant and line-mixing effects.  